## Phase - 3: Predicting Anomalies in Network Traffic (A Binary Classification problem)

### Problem Statement

discuss what architecture (how big) you do need to overfit when you have output as additional input feature

### 1. Data Load and Pre-Processing required for modeling

#### Categorical Columns

'protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login'

In [1]:
catg_cols = ['protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login']

#### Continuous Columns

In [2]:
cont_cols = [ 'duration', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

#### Target Variable

The target variable "label" contains all different types of malware attacks and also the value "normal" (i.e., not an attack) as shown below.

'normal', 'buffer_overflow', 'loadmodule', 'perl', 'neptune', 'smurf', 'guess_passwd', 'pod', 'teardrop', 'portsweep', 'ipsweep', 'land', 'ftp_write', 'back', 'imap', 'satan', 'phf', 'nmap', 'multihop', 'warezmaster', 'warezclient', 'spy', 'rootkit'

All malware attacks are grouped (transformed) to the value "abnormal" to make this problem a binary classification problem instead of a multi-class classification.


In [3]:
target_variable = 'label'

#### Data Load

In [4]:
import pandas as pd
df = pd.read_csv("../datasets/kddcup99_csv.csv")

#### Dataset size

In [5]:
df.shape

(494020, 42)

494020 records with 41 features and 1 target variable ("label") for prediction

In [6]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


As seen above, there are various type of malware attacks which can be grouped as "abnormal" to make this problem as binary classification problem

##### Group all malware attacks as "abnormal"

In [7]:
attack_types = list(df['label'].unique())
attack_types.remove('normal') # remove normal from attack types as we only want malware attacks to convert as abnormal

In [8]:
df['label'] = df['label'].replace(attack_types, 'abnormal')
df.shape

(494020, 42)

In [9]:
df['label'].value_counts()

abnormal    396743
normal       97277
Name: label, dtype: int64

In [10]:
df[catg_cols].dtypes

protocol_type     object
service           object
flag              object
land               int64
logged_in          int64
is_host_login      int64
is_guest_login     int64
dtype: object

As seen from the above datatypes of the categorical columns,the column values are not strings. We need to convert them to string before performing any analysis

##### Fixing the data types of categorical columns

In [11]:
df['protocol_type'] = df['protocol_type'].astype(str)
df['service'] = df['service'].astype(str)
df['flag'] = df['flag'].astype(str)
df['land'] = df['land'].astype(str)
df['logged_in'] = df['logged_in'].astype(str)
df['is_host_login'] = df['is_host_login'].astype(str)
df['is_guest_login'] = df['is_guest_login'].astype(str)
df[catg_cols].dtypes

protocol_type     object
service           object
flag              object
land              object
logged_in         object
is_host_login     object
is_guest_login    object
dtype: object

In [12]:
df.drop(columns=['is_host_login'], inplace=True)
df.drop(columns=['num_outbound_cmds'], inplace=True)

### Transform Target Binary label to 0 and 1

In [13]:
df['label'] = df['label'].map({'normal': 1, 'abnormal': 0})
df['label'].value_counts()

0    396743
1     97277
Name: label, dtype: int64

### One hot encode categorical columns

In [14]:
def convert_categorical_to_one_hot_encoding(dataset, column_name):
    dummy_values = pd.get_dummies(dataset[column_name])
    
    for category in dummy_values.columns:
        dummy_value_name = f"{column_name}-{category}"
        dataset[dummy_value_name] = dummy_values[category]
    dataset.drop(column_name, axis=1, inplace=True)



In [15]:
for cat_col in catg_cols:
    if cat_col != "is_host_login":
        convert_categorical_to_one_hot_encoding(df, cat_col)
df.shape

(494020, 120)

In [16]:
df.head()

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,flag-S2,flag-S3,flag-SF,flag-SH,land-0,land-1,logged_in-0,logged_in-1,is_guest_login-0,is_guest_login-1
0,0,181,5450,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
1,0,239,486,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
2,0,235,1337,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
3,0,219,1337,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0
4,0,217,2032,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,0


### Data Normalization (scaling numerical features to unit norm)

As seen from the above continous variables distribution in the section 2.6 the scale of some of the features (columns) values are not in same range as others.
Hence the features need to be normalized so that the optimization during model training will happen in a better way and the model will not be sensitive to the features.

In [17]:
cont_cols_new = [each for each in cont_cols if each!='num_outbound_cmds']

In [18]:
df_CONTCOLS_MIN = df[cont_cols_new].min(axis=0)
df_CONTCOLS_MAX = df[cont_cols_new].max(axis=0)

In [19]:
df[cont_cols_new] = (df[cont_cols_new] - df_CONTCOLS_MIN) / (df_CONTCOLS_MAX - df_CONTCOLS_MIN)

In [20]:
df[cont_cols_new].head()

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0,2.610418e-07,0.001057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.035294,0.035294,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0.0,3.446905e-07,0.000094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.074510,0.074510,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0.0,3.389216e-07,0.000259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.113725,0.113725,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0.0,3.158461e-07,0.000259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.152941,0.152941,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0.0,3.129617e-07,0.000394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.192157,0.192157,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [21]:
df.isna().values.any() # making sure there are no NaN values in the dataset

False

### Phase - 3: Using the output variable as an additional input feature

In [23]:
df['label']

0         1
1         1
2         1
3         1
4         1
         ..
494015    1
494016    1
494017    1
494018    1
494019    1
Name: label, Length: 494020, dtype: int64

In [24]:
df['output_var'] = df['label']

In [25]:
df.head()

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,flag-S3,flag-SF,flag-SH,land-0,land-1,logged_in-0,logged_in-1,is_guest_login-0,is_guest_login-1,output_var
0,0.0,2.610418e-07,0.001057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,1,1,0,1
1,0.0,3.446905e-07,0.000094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,1,1,0,1
2,0.0,3.389216e-07,0.000259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,1,1,0,1
3,0.0,3.158461e-07,0.000259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,1,1,0,1
4,0.0,3.129617e-07,0.000394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,1,1,0,1


#### 1. Shuffle DataFrame

In [26]:
df = df.sample(frac=1).reset_index(drop=True)

In [27]:
X_numpy = df.drop(columns=['label']).to_numpy()
Y_numpy = df['label'].to_numpy()

In [28]:
X_numpy.shape

(494020, 120)

In [29]:
Y_numpy.shape

(494020,)

In [30]:
# Split into training and validation, 20% validation set and 80% training 
index_20percent = int(0.2 * len(df.to_numpy()[:, 0]))
print(index_20percent)
XVALID = X_numpy[:index_20percent]
YVALID = Y_numpy[:index_20percent]
XTRAIN = X_numpy[index_20percent:]
YTRAIN = Y_numpy[index_20percent:]

98804


## discuss what architecture (how big) you do need to overfit when you have output as additional input feature

In [31]:
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [32]:
all_models = {}

In [34]:
import tensorflow as tf
metrics_to_obtain = ['accuracy', tf.keras.metrics.Precision(),
                    tf.keras.metrics.Recall(),
                    tf.keras.metrics.AUC()]

In [36]:
no_hidden_layers_model = Sequential()
no_hidden_layers_model.add(Dense(1, input_dim = len(XTRAIN[0, :])))
no_hidden_layers_model.compile(loss='binary_crossentropy', optimizer = 'rmsprop', metrics=[ 'accuracy'])

In [39]:
no_hidden_layers_model_history = no_hidden_layers_model.fit(XTRAIN, YTRAIN, validation_data = (XVALID, YVALID), 
                                            batch_size=256, epochs = 8, verbose = 1)

Epoch 1/8
1544/1544 [==============================] - 1s 959us/step - loss: 2.5568e-06 - accuracy: 1.0000 - val_loss: 6.6945e-08 - val_accuracy: 1.0000
Epoch 2/8
1544/1544 [==============================] - 1s 809us/step - loss: 2.1143e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/8
1544/1544 [==============================] - 1s 735us/step - loss: 1.8307e-06 - accuracy: 1.0000 - val_loss: 1.4081e-08 - val_accuracy: 1.0000
Epoch 4/8
1544/1544 [==============================] - 1s 840us/step - loss: 1.6096e-06 - accuracy: 1.0000 - val_loss: 2.1214e-07 - val_accuracy: 1.0000
Epoch 5/8
1544/1544 [==============================] - 1s 739us/step - loss: 1.4003e-06 - accuracy: 1.0000 - val_loss: 5.5727e-08 - val_accuracy: 1.0000
Epoch 6/8
1544/1544 [==============================] - 1s 637us/step - loss: 1.3117e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/8
1544/1544 [==============================] - 1s 665us/step - loss: 9.963